# Create climate dataset for capitals in Europe

In [1]:
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import nctoolkit as nc
%matplotlib inline

# to resolve xarray FutureWarning
xr.set_options(use_new_combine_kwarg_defaults=True)

In [2]:
# mapping between variable names in ISIMIP data and ERA5 dataset

mappings = {
    'tas': 't2m',
    'pr': 'tp',
    'prsn': 'sf',
    'sfcwind': 'si10',
    'tasmax': 'max_t2m',
    'tasmin': 'min_t2m',
}

In [3]:
# Check all available datasets first
eu_capitals = pd.read_csv("datasets/eu_capitals.csv")
print(eu_capitals.shape)
eu_capitals.head()

(58, 3)


,name,latitude,longitude
0,Mariehamn,60.00,20.00
1,Tirana,41.25,19.75
2,Andorra la Vella,42.50,1.50
3,Yerevan,40.25,44.50
4,Vienna,48.25,16.25


In [4]:
# Load datasets by 5-year periods first, then combine into 10-year periods
periods_5y = ['20110101-20151231', '20160101-20201231', '20210101-20251231', '20260101-20301231', '20310101-20351231', '20360101-20401231', '20410101-20451231', '20460101-20501231']
features = ['pr', 'prsn', 'sfcwind', 'tas', 'tasmax', 'tasmin']
datasets_5y = {}

# Load all 5-year period datasets
for feature in features:
    for period in periods_5y:
        file_path = f"cordex/rcp85/{feature}/{feature}_EUR-44_ICHEC-EC-EARTH_rcp85_r12i1p1_SMHI-RCA4_v1_day_{period}.nc"
        ds = xr.open_dataset(file_path)
        datasets_5y[f"{feature}_{period}"] = ds

# Combine into 10-year periods with all features in one dataset
periods = ['2011_2020', '2021_2030', '2031_2040', '2041_2050']
datasets = {}

# Define the mapping from 10-year periods to 5-year periods
period_mapping = {
    '2011_2020': ['20110101-20151231', '20160101-20201231'],
    '2021_2030': ['20210101-20251231', '20260101-20301231'],
    '2031_2040': ['20310101-20351231', '20360101-20401231'],
    '2041_2050': ['20410101-20451231', '20460101-20501231']
}

for period_10y, periods_5y_list in period_mapping.items():
    # Merge all features for this period
    feature_datasets = []
    
    for feature in features:
        # Combine the two 5-year periods for this feature
        ds_list = [datasets_5y[f"{feature}_{p}"] for p in periods_5y_list]
        combined_ds = xr.concat(ds_list, dim='time')
        feature_datasets.append(combined_ds)
    
    # Merge all features into one dataset
    # Use merge to combine different variables from different datasets
    datasets[period_10y] = xr.merge(feature_datasets)
    print(f"Created dataset for {period_10y} with variables: {list(datasets[period_10y].data_vars)}")

print(f"\nCombined all features into {len(periods)} 10-year period datasets")
print("Periods:", periods)
datasets.keys()

Created dataset for 2011_2020 with variables: ['pr', 'rotated_pole', 'time_bnds', 'prsn', 'sfcWind', 'tas', 'tasmax', 'tasmin']
Created dataset for 2021_2030 with variables: ['pr', 'rotated_pole', 'time_bnds', 'prsn', 'sfcWind', 'tas', 'tasmax', 'tasmin']
Created dataset for 2031_2040 with variables: ['pr', 'rotated_pole', 'time_bnds', 'prsn', 'sfcWind', 'tas', 'tasmax', 'tasmin']
Created dataset for 2041_2050 with variables: ['pr', 'rotated_pole', 'time_bnds', 'prsn', 'sfcWind', 'tas', 'tasmax', 'tasmin']

Combined all features into 4 10-year period datasets
Periods: ['2011_2020', '2021_2030', '2031_2040', '2041_2050']


dict_keys(['2011_2020', '2021_2030', '2031_2040', '2041_2050'])

In [5]:
print("Nb of latitudes:", datasets['2021_2030']['lat'].size)
print("Nb of longitudes:", datasets['2021_2030']['lon'].size)
print("Time period:", str(datasets['2021_2030']['time'].values[0])[:10], "to", str(datasets['2021_2030']['time'].values[-1])[:10])
print("Variables in dataset:", list(datasets['2021_2030'].data_vars))

Nb of latitudes: 10918
Nb of longitudes: 10918
Time period: 2021-01-01 to 2030-12-31
Variables in dataset: ['pr', 'rotated_pole', 'time_bnds', 'prsn', 'sfcWind', 'tas', 'tasmax', 'tasmin']


In [6]:
datasets['2011_2020']

<xarray.Dataset> Size: 957MB
Dimensions:       (time: 3653, rlat: 103, rlon: 106, bnds: 2)
Coordinates:
  * time          (time) datetime64[ns] 29kB 2011-01-01T12:00:00 ... 2020-12-...
  * rlat          (rlat) float64 824B -23.21 -22.77 -22.33 ... 20.79 21.23 21.67
  * rlon          (rlon) float64 848B -28.21 -27.77 -27.33 ... 17.11 17.55 17.99
    lat           (rlat, rlon) float64 87kB ...
    lon           (rlat, rlon) float64 87kB ...
    height        float64 8B ...
Dimensions without coordinates: bnds
Data variables:
    pr            (time, rlat, rlon) float32 160MB 0.0 0.0 ... 0.000119
    rotated_pole  |S1 1B ...
    time_bnds     (time, bnds) datetime64[ns] 58kB 2011-01-01 ... 2021-01-01
    prsn          (time, rlat, rlon) float32 160MB 0.0 0.0 ... 0.000119
    sfcWind       (time, rlat, rlon) float32 160MB 5.093 5.281 ... 6.855 6.152
    tas           (time, rlat, rlon) float32 160MB 284.9 285.4 ... 259.7 258.2
    tasmax        (time, rlat, rlon) float32 160MB 292.8 293.3 ... 267.2 266.2
    tasmin        (time, rlat, rlon) float32 160MB 279.1 279.5 ... 252.4 248.3
Attributes: (12/22)
    Conventions:                    CF-1.4
    contact:                        rossby.cordex@smhi.se
    creation_date:                  2013-06-15-T20:38:46Z
    experiment:                     RCP8.5
    experiment_id:                  rcp85
    driving_experiment:             ICHEC-EC-EARTH, rcp85, r12i1p1
    ...                             ...
    product:                        output
    references:                     http://www.smhi.se/en/Research/Research-d...
    tracking_id:                    c648913e-c8f3-4f97-848c-a48438dc2962
    rossby_comment:                 201211: CORDEX Europe 0.44 deg | RCA4 v1 ...
    rossby_run_id:                  201211
    rossby_grib_path:               /nobackup/rossby16/rossby/joint_exp/corde...

## Convert units to match ERA5 dataset (precipitation and snowfall from kg m-2 s-1 to m)

#### Units comparison between ERA5 and ISIMIP3b:

| Feature             | ERA5 Unit  | ISIMIP3b Unit |
| ------------------- | ---------- | ------------- |
| 2m Temperature      | Kelvin (K) | K             |
| wind speed          | m s^-1    | m s^-1        |
| total precipitation | m          | kg m^-2 s^-1  |
| snowfall            | m          | kg m^-2 s^-1  |

- 1 kg m⁻² = 1 mm water
- precipitation rate is for one day -> multiply by 86400 (seconds in a day) and convert mm to m (divide by 1000)

In [9]:
def convert_precipitation_units(dataset):
    """Convert precipitation and snowfall from kg m-2 s-1 to m"""
    dataset['pr'] = dataset['pr'] * 86400 / 1000 
    dataset['prsn'] = dataset['prsn'] * 86400 / 1000 
    return dataset

In [11]:
datasets = {period: convert_precipitation_units(ds) for period, ds in datasets.items()}

In [12]:
datasets['2011_2020']

<xarray.Dataset> Size: 957MB
Dimensions:       (time: 3653, rlat: 103, rlon: 106, bnds: 2)
Coordinates:
  * time          (time) datetime64[ns] 29kB 2011-01-01T12:00:00 ... 2020-12-...
  * rlat          (rlat) float64 824B -23.21 -22.77 -22.33 ... 20.79 21.23 21.67
  * rlon          (rlon) float64 848B -28.21 -27.77 -27.33 ... 17.11 17.55 17.99
    lat           (rlat, rlon) float64 87kB ...
    lon           (rlat, rlon) float64 87kB ...
    height        float64 8B ...
Dimensions without coordinates: bnds
Data variables:
    pr            (time, rlat, rlon) float32 160MB 0.0 0.0 ... 0.005696 0.01028
    rotated_pole  |S1 1B ...
    time_bnds     (time, bnds) datetime64[ns] 58kB 2011-01-01 ... 2021-01-01
    prsn          (time, rlat, rlon) float32 160MB 0.0 0.0 ... 0.005696 0.01028
    sfcWind       (time, rlat, rlon) float32 160MB 5.093 5.281 ... 6.855 6.152
    tas           (time, rlat, rlon) float32 160MB 284.9 285.4 ... 259.7 258.2
    tasmax        (time, rlat, rlon) float32 160MB 292.8 293.3 ... 267.2 266.2
    tasmin        (time, rlat, rlon) float32 160MB 279.1 279.5 ... 252.4 248.3
Attributes: (12/22)
    Conventions:                    CF-1.4
    contact:                        rossby.cordex@smhi.se
    creation_date:                  2013-06-15-T20:38:46Z
    experiment:                     RCP8.5
    experiment_id:                  rcp85
    driving_experiment:             ICHEC-EC-EARTH, rcp85, r12i1p1
    ...                             ...
    product:                        output
    references:                     http://www.smhi.se/en/Research/Research-d...
    tracking_id:                    c648913e-c8f3-4f97-848c-a48438dc2962
    rossby_comment:                 201211: CORDEX Europe 0.44 deg | RCA4 v1 ...
    rossby_run_id:                  201211
    rossby_grib_path:               /nobackup/rossby16/rossby/joint_exp/corde...

## Keep data within a radius of 50km (0.45 degrees) around each capital

In [13]:
# Haversine is used to calculate distance between two lat/lon points in km, it is the preferred method for geographical coordinates

def haversine_vectorized(lat_grid, lon_grid, lat_city, lon_city):
    """
    Vectorized haversine calculation for entire grid.
    Returns distances in km between each grid point and the city.
    
    Parameters:
    -----------
    lat_grid : ndarray
        2D array of latitudes (rlat, rlon)
    lon_grid : ndarray
        2D array of longitudes (rlat, rlon)
    lat_city : float
        City latitude
    lon_city : float
        City longitude
    
    Returns:
    --------
    distances : ndarray
        2D array of distances in km
    """
    R = 6371.0  # Earth radius in km
    
    # Convert to radians
    lat_grid_rad = np.deg2rad(lat_grid)
    lon_grid_rad = np.deg2rad(lon_grid)
    lat_city_rad = np.deg2rad(lat_city)
    lon_city_rad = np.deg2rad(lon_city)
    
    # Haversine formula
    dlat = lat_grid_rad - lat_city_rad
    dlon = lon_grid_rad - lon_city_rad
    
    a = np.sin(dlat / 2)**2 + np.cos(lat_grid_rad) * np.cos(lat_city_rad) * np.sin(dlon / 2)**2
    distances = 2 * R * np.arcsin(np.sqrt(a))
    
    return distances


def precompute_city_indices(dataset, cities_df, radius_km=50):
    """
    Pre-compute grid point indices for all cities once.
    This can be reused across all time periods since the grid is the same.
    Stores actual indices instead of masks for faster indexing.
    
    Parameters:
    -----------
    dataset : xr.Dataset
        Any dataset with the same spatial grid (rlat, rlon, lat, lon)
    cities_df : pd.DataFrame
        DataFrame with columns: name, latitude, longitude
    radius_km : float
        Radius in km around each city
    
    Returns:
    --------
    city_info_list : list of dict
        List of dictionaries with keys: 'name', 'rlat_indices', 'rlon_indices', 'lat', 'lon'
    """
    lat_grid = dataset['lat'].values
    lon_grid = dataset['lon'].values
    
    city_info_list = []
    
    print(f"Pre-computing grid indices for {len(cities_df)} cities...")
    
    for _, row in cities_df.iterrows():
        city = row['name']
        lat_city = row['latitude']
        lon_city = row['longitude']
        
        # Quick bounding box filter
        buffer_deg = radius_km / 111.0
        bbox_mask = ((lat_grid >= lat_city - buffer_deg) & 
                     (lat_grid <= lat_city + buffer_deg) &
                     (lon_grid >= lon_city - buffer_deg) & 
                     (lon_grid <= lon_city + buffer_deg))
        
        bbox_mask = bbox_mask.astype(bool)
        
        if not np.any(bbox_mask):
            print(f"  Warning: No grid points found for {city}. Skipping.")
            continue
        
        # Precise haversine calculation
        distances = haversine_vectorized(lat_grid, lon_grid, lat_city, lon_city)
        distance_mask = (distances <= radius_km).astype(bool)
        
        # Final mask
        final_mask = bbox_mask & distance_mask
        
        if not np.any(final_mask):
            print(f"  Warning: No grid points within {radius_km}km for {city}. Skipping.")
            continue
        
        # Extract indices where mask is True
        rlat_indices, rlon_indices = np.where(final_mask)
        
        city_info_list.append({
            'name': city,
            'rlat_indices': rlat_indices,
            'rlon_indices': rlon_indices,
            'lat': lat_city,
            'lon': lon_city
        })
    
    print(f"Successfully pre-computed indices for {len(city_info_list)} cities.")
    return city_info_list


def apply_city_indices(dataset, city_info_list):
    """
    Apply pre-computed city indices to a dataset.
    Ultra-fast: directly indexes the data without creating intermediate masked arrays.
    
    Parameters:
    -----------
    dataset : xr.Dataset
        Dataset with rlat, rlon dimensions
    city_info_list : list of dict
        Pre-computed indices from precompute_city_indices()
    
    Returns:
    --------
    result : xr.Dataset
        Dataset with city dimension containing all cities
    """
    city_data_list = []
    
    # List of climate variables to process (exclude metadata variables)
    climate_vars = ['pr', 'prsn', 'sfcWind', 'tas', 'tasmax', 'tasmin']
    vars_to_process = [v for v in climate_vars if v in dataset.data_vars]
    
    for city_info in city_info_list:
        city_name = city_info['name']
        rlat_idx = city_info['rlat_indices']
        rlon_idx = city_info['rlon_indices']
        
        # Create a dictionary to store averaged data for each variable
        city_vars = {}
        
        # For each climate variable in the dataset
        for var_name in vars_to_process:
            # Get the data array
            data = dataset[var_name].values  # shape: (time, rlat, rlon)
            
            # Check if data has the expected shape
            if data.ndim != 3:
                print(f"  Warning: Skipping {var_name} - unexpected dimensions: {data.shape}")
                continue
            
            # Extract only the points within the radius using fancy indexing
            # This creates an array of shape (time, n_points)
            selected_points = data[:, rlat_idx, rlon_idx]
            
            # Average over the spatial points for each time step
            averaged = np.nanmean(selected_points, axis=1)  # shape: (time,)
            
            city_vars[var_name] = averaged
        
        # Create a dataset for this city
        city_ds = xr.Dataset(
            {var: (['time'], data) for var, data in city_vars.items()},
            coords={'time': dataset['time']}
        )
        city_ds = city_ds.assign_coords(city=city_name)
        city_data_list.append(city_ds)
    
    # Concatenate all cities
    result = xr.concat(city_data_list, dim='city')
    
    # Add city coordinates
    result = result.assign_coords({
        'latitude': ('city', [c['lat'] for c in city_info_list]),
        'longitude': ('city', [c['lon'] for c in city_info_list])
    })
    
    return result


In [14]:
from haversine import haversine as haversine_lib

paris = (48.8567, 2.3508)
lausanne = (46.5197, 6.6323)
print("With library:", haversine_lib(lausanne, paris))
print("With custom function:", haversine_vectorized(46.5197, 6.6323, 48.8567, 2.3508))

With library: 412.4865729527725
With custom function: 412.4860032028388


In [15]:
# Pre-compute grid indices once (they're the same for all periods since the grid is identical)
print("Pre-computing spatial indices for all cities (done once)")
city_info = precompute_city_indices(
    datasets[periods[0]],  # Use first period as reference for the grid
    eu_capitals, 
    radius_km=50
)

print("Applying indices to all periods")
ds_europe = {}

for period in periods:
    print(f"  Processing period {period}...", end=' ')
    ds_europe[period] = apply_city_indices(datasets[period], city_info)
    print(f"done (time={ds_europe[period].sizes['time']}, cities={ds_europe[period].sizes['city']})")

Pre-computing spatial indices for all cities (done once)
Pre-computing grid indices for 58 cities...
Successfully pre-computed indices for 54 cities.
Applying indices to all periods
  Processing period 2011_2020... done (time=3653, cities=54)
  Processing period 2021_2030... done (time=3652, cities=54)
  Processing period 2031_2040... done (time=3653, cities=54)
  Processing period 2041_2050... done (time=3652, cities=54)


In [16]:
ds_europe['2011_2020']

<xarray.Dataset> Size: 5MB
Dimensions:    (city: 54, time: 3653)
Coordinates:
  * city       (city) <U16 3kB 'Mariehamn' 'Tirana' ... 'London' 'North Nicosia'
  * time       (time) datetime64[ns] 29kB 2011-01-01T12:00:00 ... 2020-12-31T...
    height     float64 8B ...
    latitude   (city) float64 432B 60.0 41.25 42.5 48.25 ... 50.5 51.5 35.25
    longitude  (city) float64 432B 20.0 19.75 1.5 16.25 ... 30.5 -0.0 33.25
Data variables:
    pr         (city, time) float32 789kB 0.0002266 0.001051 ... 0.0004566
    prsn       (city, time) float32 789kB 0.0 0.0 0.0 0.0 ... 0.0 2.403e-05 0.0
    sfcWind    (city, time) float32 789kB 5.377 11.76 7.861 ... 4.712 2.288
    tas        (city, time) float32 789kB 274.2 271.6 270.8 ... 286.8 285.5
    tasmax     (city, time) float32 789kB 274.9 274.0 272.0 ... 289.2 289.3
    tasmin     (city, time) float32 789kB 273.0 269.8 269.8 ... 284.1 281.5

## Aggregate daily data to seasonal data

In [17]:
def aggregate_monthly(dataset):
    """Aggregate daily data to monthly data by taking the mean over each month."""
    monthly_data = dataset.resample(time='ME').mean()
    return monthly_data

def aggregate_per_season(dataset):
    """Aggregate daily data to seasonal data by taking the mean over each season."""
    seasonal_data = dataset.resample(time='QS-DEC').mean()
    return seasonal_data

In [18]:
ds_seasonal = {}

for period in periods:
    ds_seasonal[period] = aggregate_per_season(ds_europe[period])

In [19]:
ds_seasonal['2011_2020']

<xarray.Dataset> Size: 58kB
Dimensions:    (time: 41, city: 54)
Coordinates:
  * time       (time) datetime64[ns] 328B 2010-12-01 2011-03-01 ... 2020-12-01
  * city       (city) <U16 3kB 'Mariehamn' 'Tirana' ... 'London' 'North Nicosia'
    height     float64 8B 10.0
    latitude   (city) float64 432B 60.0 41.25 42.5 48.25 ... 50.5 51.5 35.25
    longitude  (city) float64 432B 20.0 19.75 1.5 16.25 ... 30.5 -0.0 33.25
Data variables:
    pr         (time, city) float32 9kB 0.002041 0.001061 ... 0.005209 0.002082
    prsn       (time, city) float32 9kB 8.711e-05 1.981e-05 ... 6.922e-06
    sfcWind    (time, city) float32 9kB 7.645 4.8 5.306 ... 5.06 6.748 3.761
    tas        (time, city) float32 9kB 275.5 278.1 273.7 ... 275.0 278.8 286.0
    tasmax     (time, city) float32 9kB 276.5 282.0 276.1 ... 277.2 280.7 289.4
    tasmin     (time, city) float32 9kB 274.3 275.0 271.3 ... 272.6 276.6 282.8

In [24]:
# Verification
print(f"Before seasonal aggregation: {ds_europe['2011_2020']['time'].shape[0]}")
print(f"After seasonal aggregation: {ds_seasonal['2011_2020']['time'].shape[0]}")

Before seasonal aggregation: 3653
After seasonal aggregation: 41


# Average climate features for each decade

For each feature, for each capital (lat,lon), do the mean on 10 years for each month. For example, for the feature "2m temperature" (t2m), for the capital of France (48.8566, 2.3522), do the mean of all January data from 1970 to 1979, then do the mean of all February data from 1970 to 1979, and so on until December. Then repeat the same operation for the periods 1980-1989, 1990-1999, 2000-2009 and 2010-2020. In the end, we will have for each year, a dataset with 2 + 12 * 4 = 50 columns: latitude, longitude, and for each month (12) the 4 features (sf, tp, t2m, si10).

<img src="images/ds_columns_past_v2.png" style="width:1000px;"  />


In [54]:
def average_over_period(dataset, month) -> xr.Dataset:
    """Average the dataset over a specified time period."""
    period_data = dataset.sel(time=dataset['time.month'] == month)
    averaged_data = period_data.mean(dim='time')
    return averaged_data

In [51]:
ds_seasonal['2011_2020'].sel(time=ds_seasonal['2011_2020']['time.month'] == 3)

<xarray.Dataset> Size: 17kB
Dimensions:    (time: 10, city: 54)
Coordinates:
  * time       (time) datetime64[ns] 80B 2011-03-01 2012-03-01 ... 2020-03-01
  * city       (city) <U16 3kB 'Mariehamn' 'Tirana' ... 'London' 'North Nicosia'
    height     float64 8B 10.0
    latitude   (city) float64 432B 60.0 41.25 42.5 48.25 ... 50.5 51.5 35.25
    longitude  (city) float64 432B 20.0 19.75 1.5 16.25 ... 30.5 -0.0 33.25
Data variables:
    pr         (time, city) float32 2kB 0.002044 0.0008252 ... 0.00225 0.001742
    prsn       (time, city) float32 2kB 1.208e-05 1.371e-05 ... 6.572e-05
    sfcWind    (time, city) float32 2kB 6.034 4.361 4.81 ... 4.25 4.871 4.105
    tas        (time, city) float32 2kB 278.6 286.2 278.9 ... 282.3 281.5 289.9
    tasmax     (time, city) float32 2kB 279.9 291.4 282.3 ... 286.1 284.3 294.5
    tasmin     (time, city) float32 2kB 277.1 280.9 275.6 ... 278.2 278.5 284.8

In [66]:
ds_periods = {}
months = [12, 3, 6, 9]  # winter, spring, summer, autumn

for period in periods:
    ds_season = ds_seasonal[period]
    ds_result = xr.Dataset()
    for month in months:
        ds_month = average_over_period(
            ds_season,
            month
        )
        if ds_result.sizes:
            ds_result = xr.concat([ds_result, ds_month], dim='time')
        else:
            ds_result = ds_month
    ds_periods[period] = ds_result
    

In [68]:
ds_periods['2011_2020']['tas']

<xarray.DataArray 'tas' (time: 4, city: 54)> Size: 864B
array([[274.89578, 279.35266, 274.13293, 273.95682, 271.19974, 277.76083,
        273.36533, 273.50226, 276.1427 , 284.27267, 273.97855, 276.20212,
        272.17834, 277.51892, 271.67825, 278.47815, 275.17133, 284.55884,
        282.53943, 281.90018, 279.79688, 274.5554 , 271.62152, 278.82764,
        278.91208, 279.79688, 281.07462, 274.46646, 273.32455, 270.31262,
        271.5637 , 276.37448, 274.68954, 286.96335, 273.7372 , 277.3756 ,
        275.01013, 278.1085 , 268.74728, 273.2682 , 284.59616, 275.356  ,
        268.26553, 279.39874, 276.9378 , 274.26407, 274.21622, 277.20493,
        272.86743, 273.35205, 273.4448 , 271.91077, 278.47174, 284.27267],
       [277.47836, 284.6072 , 277.52197, 281.74884, 279.4405 , 281.89056,
        279.09183, 280.2239 , 283.33176, 289.34283, 280.8373 , 280.07175,
        277.43912, 279.60257, 277.07834, 282.49466, 281.46567, 287.94678,
        287.66858, 282.677  , 284.4964 , 282.834  , 276.39645, 280.96976,
        280.8094 , 284.4964 , 282.6685 , 281.02914, 278.20663, 275.4956 ,
        279.22852, 281.03687, 281.21664, 288.7879 , 282.69662, 281.5927 ,
        280.0927 , 281.85016, 275.26193, 280.891  , 287.25946, 284.35147,
        277.80145, 284.27917, 284.4754 , 282.47833, 280.5722 , 282.7021 ,
        278.15994, 278.21475, 280.6289 , 281.26855, 281.59732, 289.34283],
       [286.60052, 295.8371 , 286.09018, 291.32626, 288.6936 , 288.84802,
        288.39713, 290.66467, 294.38297, 299.1011 , 289.53424, 288.0834 ,
        287.13858, 284.40137, 287.09805, 290.18546, 289.43024, 295.80417,
        298.0039 , 288.32257, 294.88437, 293.20038, 282.45673, 286.62415,
        286.59366, 294.88437, 288.80548, 292.03677, 287.56082, 285.07352,
        288.16147, 288.30957, 292.5522 , 295.7553 , 294.21368, 291.02167,
        290.47025, 288.6196 , 284.18225, 289.28485, 293.67612, 296.4826 ,
        288.71048, 295.35812, 296.1068 , 292.2878 , 290.26657, 294.72797,
        287.181  , 287.05246, 292.51724, 291.67242, 288.29364, 299.1011 ],
       [282.7422 , 286.70068, 280.2283 , 282.37372, 280.2113 , 283.76004,
        280.33554, 281.19775, 283.9887 , 292.19708, 281.81818, 283.65314,
        280.80585, 281.70938, 280.64267, 284.4542 , 282.8006 , 291.10748,
        289.76202, 286.55508, 286.8096 , 283.36502, 277.59326, 283.80554,
        284.12793, 286.8096 , 286.01544, 281.89728, 281.59125, 277.99564,
        280.29037, 282.3913 , 282.15118, 292.8002 , 283.72894, 283.98917,
        281.87872, 284.29083, 278.1296 , 281.78992, 290.5075 , 284.75323,
        278.64227, 286.4089 , 285.0153 , 282.94028, 281.58295, 285.19492,
        281.09406, 280.27972, 282.3434 , 281.97137, 284.1503 , 292.19708]],
      dtype=float32)
Coordinates:
  * city       (city) <U16 3kB 'Mariehamn' 'Tirana' ... 'London' 'North Nicosia'
    height     float64 8B 10.0
    latitude   (city) float64 432B 60.0 41.25 42.5 48.25 ... 50.5 51.5 35.25
    longitude  (city) float64 432B 20.0 19.75 1.5 16.25 ... 30.5 -0.0 33.25
Dimensions without coordinates: time

In [67]:
print(ds_periods.keys())

dict_keys(['2011_2020', '2021_2030', '2031_2040', '2041_2050'])


## Reconstruct the dataframe with all features for all decades

In [47]:
def aggregate_feature_per_period(features, seasons, period, datasets) -> pd.DataFrame:
    """Aggregate features per specified seasons and return a DataFrame."""
    records = []
    
    # Récupérer les villes et coordonnées depuis le premier dataset
    first_ds = datasets[f"{features[0]}_{period}_{seasons[0]}"]
    cities = first_ds['city'].values
    lats = first_ds['latitude'].values
    lons = first_ds['longitude'].values
    
    # Créer une ligne par ville
    for i, city in enumerate(cities):
        record = {
            'city': city,
            'latitude': lats[i],
            'longitude': lons[i]
        }
        
        # Ajouter toutes les features pour toutes les saisons
        for feature in features:
            for season in seasons:
                ds_key = f"{feature}_{period}_{season}"
                ds = datasets[ds_key]
                
                # Extraire la valeur pour cette ville (index i)
                value = ds[feature].values[i]
                record[f'{mappings[feature]}_{season}'] = value
        
        records.append(record)
    
    df = pd.DataFrame(records)
    return df

In [48]:
final_dfs = {}

for period in periods:
    df_period = aggregate_feature_per_period(features, seasons, period, ds_periods)
    final_dfs[period] = df_period

In [49]:
final_dfs['2021_2030'].head()

,city,latitude,longitude,tp_winter,tp_spring,tp_summer,tp_autumn,sf_winter,sf_spring,sf_summer,...,t2m_summer,t2m_autumn,max_t2m_winter,max_t2m_spring,max_t2m_summer,max_t2m_autumn,min_t2m_winter,min_t2m_spring,min_t2m_summer,min_t2m_autumn
0,Mariehamn,60.00,20.00,0.002015,0.001961,0.001964,0.002003,0.000391,0.000369,0.000361,...,280.611969,280.481140,281.783142,281.797516,282.117676,281.976410,278.607880,278.598907,278.893616,278.782898
1,Tirana,41.25,19.75,0.002858,0.002776,0.002699,0.002765,0.000060,0.000059,0.000059,...,288.788422,288.582031,293.796936,293.956879,294.304688,294.080994,283.083435,283.154541,283.410645,283.255127
2,Andorra la Vella,42.50,1.50,0.002316,0.002333,0.002315,0.002277,0.000501,0.000479,0.000466,...,283.121796,282.931030,287.733246,287.869629,288.242767,288.029785,277.738007,277.765137,278.066376,277.916046
3,Yerevan,40.25,44.50,0.001556,0.001566,0.001518,0.001519,0.000587,0.000549,0.000531,...,282.407501,282.096405,287.863159,288.292480,288.706421,288.374268,275.837860,276.215271,276.521149,276.237396
4,Vienna,48.25,16.25,0.002194,0.002212,0.002226,0.002186,0.000221,0.000201,0.000196,...,284.754761,284.471008,288.087616,288.275085,288.631287,288.296936,280.479340,280.564148,280.862915,280.640594


In [50]:
# Check for any missing values in each period dataset
for period_name, df in final_dfs.items():
    missing_counts = df.isnull().sum()
    total_missing = missing_counts.sum()
    print(f"\n{period_name}:")
    print(f"  Total missing values: {total_missing}")
    if total_missing > 0:
        print("  Missing values per column:")
        print(missing_counts[missing_counts > 0])
        print("Cities with missing values:")
        print(df[df.isnull().any(axis=1)]['city'].values)


2011_2014:
  Total missing values: 0

2021_2030:
  Total missing values: 0

2031_2040:
  Total missing values: 0

2041_2050:
  Total missing values: 0


## Save the datasets as CSV files

In [51]:
# Download the datasets as CSV files
for period_name, df in final_dfs.items():
    df.to_csv(f"datasets/climate_features_{period_name}_future.csv", index=False)
    print(f"Saved climate_features_{period_name}_future.csv")

Saved climate_features_2011_2014_future.csv
Saved climate_features_2021_2030_future.csv
Saved climate_features_2031_2040_future.csv
Saved climate_features_2041_2050_future.csv
